### Task4 基于深度学习的文本分类1-fastText
参考1链接 https://tianchi.aliyun.com/notebook-ai/detail?spm=5176.12586969.1002.15.6406111aIKCSLV&postId=118255
参考2链接 https://blog.csdn.net/weixin_45415853/article/details/107549391

   与传统机器学习不同，深度学习既提供特征提取功能，也可以完成分类的功能。从本章开始我们将学习如何使用深度学习来完成文本表示。



### 1 学习目标
   《1》学习FastText的使用和基础原理
   《2》学会使用验证集进行调参



### 2 文本表示方法 Part2


#### 2-1 现有文本表示方法的缺陷
在上一章节，我们介绍几种文本表示方法：
    One-hot
    Bag of Words
    N-gram
    TF-IDF

    也通过sklean进行了相应的实践，相信你也有了初步的认知。但上述方法都或多或少存在一定的问题：转换得到的向量维度很高，需要较长的训练实践；没有考虑单词与单词之间的关系，只是进行了统计。
    与这些表示方法不同，深度学习也可以用于文本表示，还可以将其映射到一个低纬空间。其中比较典型的例子有：FastText、Word2Vec和Bert。在本章我们将介绍FastText，将在后面的内容介绍Word2Vec和Bert。
 
 
#### 2-2 n-gram文档向量

##### 2-2-1 word-n-gram
   在文本特征提取中，常常能看到n-gram的身影。它是一种基于语言模型的算法，基本思想是将文本内容按照字节顺序进行大小为N的滑动窗口操作，最终形成长度为N的字节片段序列。看下面的例子：
   例句： 我来到达观数据参观
   相应的bigram特征为：我来 来到 到达 达观 观数 数据 据参 参观
   相应的trigram特征为：我来到 来到达 到达观 达观数 观数据 数据参 据参观
   
注意一点：n-gram中的gram根据粒度不同，有不同的含义。它可以是字粒度，也可以是词粒度的。上面所举的例子属于字粒度的n-gram，词粒度的n-gram看下面例子：
   例句： 我 来到 达观数据 参观
   相应的bigram特征为：我/来到   来到/达观数据   达观数据/参观
   相应的trigram特征为：我/来到/达观数据   来到/达观数据/参观

##### 2-2-2 char-n-gram
   word2vec把语料库中的每个单词当成原子的，它会为每个单词生成一个向量。这忽略了单词内部的形态特征，比如：“apple” 和“apples”，“达观数据”和“达观”，这两个例子中，两个单词都有较多公共字符，即它们的内部形态类似，但是在传统的word2vec中，这种单词内部形态信息因为它们被转换成不同的id丢失了。
   为克服此问题，fastText使用字符级别n-grams表示一个单词。对于单词“apple”，假设n取值3，则它的trigram有“<ap”, “app”, “ppl”, “ple”, “le>”
   其中，<表示前缀，>表示后缀。于是，可以用这些trigram表示“apple”这个单词，进一步，可以用这5个trigram的向量叠加表示“apple”的词向量。

这带来两点好处：
   《1》对于低频词生成的词向量效果会更好。因为它们的n-gram可以和其它词共享。
   《2》对于训练词库之外的单词，仍然可以构建它们的词向量。我们可以叠加它们的字符级n-gram向量。


#### 2-3 FastText
   FastText是一种典型的深度学习词向量的表示方法，它非常简单通过Embedding层将单词映射到稠密空间，然后将句子中所有的单词在Embedding空间中进行平均，进而完成分类操作。
   所以FastText是一个三层的神经网络，输入层、隐含层和输出层。
![avatar](C://Users/hs/Desktop/nlp/data/FastText01.jpg)
   将整篇文档的词及n-gram向量叠加平均得到文档向量，然后使用文档向量做softmax多分类。

##### 2-3-1 模型结构
   此架构图没有展示词向量的训练过程。可以看到，fastText模型只有三层：输入层、隐含层、输出层。输入是多个单词及其n-gram的词向量表示。隐含层都是对多个词向量的叠加平均。输出是文档对应的类标。

##### 2-3-3  hierarchical softmax
   hierarchical softmax是一个损失函数，当标签类很大时，通过层级搜索来使softmax整体计算加快。
   想法是通过构建一个二叉树，其叶子对应于标签，每一个中间节点有一个二元决策激活（例如sigmoid），预测是应该向左还是向右。输出单元（叶子）的概率由沿着从根到输出单元的路径的中间节点的概率的乘积给出。 
   在fastText中，使用Huffman tree，对于更频繁的输出，查找时间更快，因此输出的平均查找时间是最佳的。

如下是使用keras实现的FastText网络结构：

In [ ]:
## FastText实践案例
## coding:tf-8
from __future__ import unicode_literals 

from keras.models import Sequential 
from keras.layers import Embedding 
from keras.layers import GlobalAveragePooling1D 
from keras.layers import Dense


VOCAB_SIZE = 2000 
EBIDDNG_DM = 100 
MAX_WORDS = 500
CLASS_NUM = 5


def bui1d_fastText():
    model = Sequential()
    # 通过embedding层，我们将词汇映射成 EMBEDDING_DIM 维向量
    model.add( Embedding( VOCAB_SIZE , EBIDDNG_DM ,input_length=MAX_WORDS ))
    # 通过GlobalAveragePooling10，我们平均了文档中所有词的embedding 
    model.add( GlobalAveragePooling1D() )
    ##通过输出层Softmox分类（真实的fastText这里是分层Softmox），得到类别概事分布
    model.add( Dense( CLASS_NUM, activation='softmax'))
    ##定义损庆a段、优化器、分实度量指坛
    model.compile( loss= "categorical_crossentropy", optimizer='SGD',metrics=['accuracy'])
    return modeal


if __name__ == ' ratn':
    model = build_fastText()
    print(model.summary())

#### FastText在文本分类任务上，是优于TF-IDF的：
   FastText用单词的Embedding叠加获得的文档向量，将相似的句子分为一类
   FastText学习到的Embedding空间维度比较低，可以快速进行训练
如果想深度学习，可以参考论文：Bag of Tricks for Efficient Text Classification, 地址 https://arxiv.org/abs/1607.01759


#### 2-4 实践：基于FastText文本分类
FastText可以快速的在CPU上进行训练，最好的实践方法就是官方开源的版本： https://github.com/facebookresearch/fastText/tree/master/python
《1》pip安装   pip install fasttext
《2》源码安装
    git clone https://github.com/facebookresearch/fastText.git
    cd fastText
    sudo pip install .
两种安装方法都可以安装，如果你是初学者可以优先考虑使用pip安装。
//代码如下
此时数据量比较小得分为0.82，当不断增加训练集数量时，FastText的精度也会不断增加5w条训练样本时，验证集得分可以到0.89-0.90左右。

In [ ]:
## 实践案例1
import pandas as pd
from sklearn.metrics import f1_score

# 转换为FastText需要的格式
train_df = pd.read_csv('../data/train_set.csv', sep='\t', nrows=15000)
train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
train_df[['text','label_ft']].iloc[:-5000].to_csv('train.csv', index=None, header=None, sep='\t')


import fasttext
model = fasttext.train_supervised('train.csv', lr=1.0, wordNgrams=2, verbose=2, minCount=1, epoch=25, loss="hs")

val_pred = [ model.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[-5000:]['text'] ]
print(f1_score(train_df['label'].values[-5000:].astype(str), val_pred, average='macro'))
# 0.82

In [ ]:
## fasttext.supervised() 参数
##（1）用法：
import fasttext
model = fasttext.train_supervised(  
    input_fine,                                
    lr=1.0,                                
    dim=100,                                 
    ws=5,                               
    epoch=25,                               
    minCount=1,                               
    wordNgrams=2, loss='ns', 
    bucket=2000000,                        
    thread=12, 
    lrUpdateRate=100, 
    t=1e-4,                     
    label='__label__', 
    verbose=2
)


##（2）参数说明：
''' 
input_file                 训练文件路径（必须）
output                     输出文件路径
label_prefix               标签前缀 default __label__
lr                         学习率 default 0.1
lr_update_rate             学习率更新速率 default 100
dim                        词向量维度 default 100
ws                         上下文窗口大小 default 5
epoch                      epochs 数量 default 5
min_count                  最低词频，过滤词频低的词 default 5
word_ngrams                n-gram 设置 default 1
loss                       损失函数 {ns,hs,softmax,ova}
minn                       最小字符长度 default 3，用于设定word-n-gram
maxn                       最大字符长度 default 6, 用于设定word-g-gram
thread                     线程数量 default 12
t                          采样阈值 default 0.0001
silent                     禁用 c++ 扩展日志输出 default 1
encoding                   指定 input_file 编码 default utf-8
verbose                    日志显示，0不显示，1显示进度条，2显示详细信息
pretrained_vectors         指定使用已有的词向量 .vec 文件 default None

'''


## loss损失函数:
'''
    ns: negtive sample，含义是普通的softmax方法，再输出层每次再保留目标单词的同时，不采用所有的词，而是仅激活其中的一部分（比如100个）单词同目标单词 作为输出层维度（总词数）。
    hs: hierarchical softmax
    ova: one-vs-all，用于多分类
'''


import pandas as pd
from sklearn.metrics import f1_score
import fasttext

# 转换为FastText需要的格式
train_df = pd.read_csv('TRAIN_DATA/train_set.csv', sep='\t', nrows=15000)
train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
train_df[['text','label_ft']].iloc[:-5000].to_csv('fasttext_train.csv', index=None, header=None, sep='\t')


model = fasttext.train_supervised('fasttext_train.csv', lr=1.0, wordNgrams=2, verbose=2, minCount=1, epoch=25, loss='ova')
x=train_df.iloc[-5000:]['text']
y=train_df['label']
val_pred = [model.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[-5000:]['text']]
val_y0 = train_df['label'].values[-5000:]
val_y1 = val_y0.astype(str)
print(f1_score(train_df['label'].values[-5000:].astype(str), val_pred, average='macro'))

## // 0.8845029863261721


#### 2-5 如何使用验证集调参
在使用TF-IDF和FastText中，有一些模型的参数需要选择，这些参数会在一定程度上影响模型的精度，那么如何选择这些参数呢？
   《1》通过阅读文档，要弄清楚这些参数的大致含义，那些参数会增加模型的复杂度
   《2》通过在验证集上进行验证模型精度，找到模型在是否过拟合还是欠拟合

此时数据量比较小得分为0.82，当不断增加训练集数量时，FastText的精度也会不断增加5w条训练样本时，验证集得分可以到0.89-0.90左右。

这里我们使用10折交叉验证，每折使用9/10的数据进行训练，剩余1/10作为验证集检验模型的效果。这里需要注意每折的划分必须保证标签的分布与整个数据集的分布一致。

label2id = {}
for i in range(total):
    label = str(all_labels[i])
    if label not in label2id:
        label2id[label] = [i]
    else:
        label2id[label].append(i)

通过10折划分，我们一共得到了10份分布一致的数据，索引分别为0到9，每次通过将一份数据作为验证集，剩余数据作为训练集，获得了所有数据的10种分割。不失一般性，我们选择最后一份完成剩余的实验，即索引为9的一份做为验证集，索引为1-8的作为训练集，然后基于验证集的结果调整超参数，使得模型性能更优。


##### 2-5-1 十折交叉验证
(1) 留出法 Hold-out method
将原始数据随机分为两组,一组做为训练集,一组做为验证集,利用训练集训练分类器,然后利用验证集验证模型,记录最后的分类准确率

(2)十折交叉验证 10-fold cross-validation
将数据集分成十份，轮流将其中9份作为训练数据，1份作为测试数据，进行试验。

每次试验都会得出相应的正确率(或差错率)。10次的结果的正确率(或差错率)的平均值作为对算法精度的估计，一般还需要进行多次10折交叉验证(例如10次10折交叉验证)，再求其均值，作为对算法准确性的估计。

(3) 留一验证 leave-one-out cross-validation
留一验证（LOOCV）意指只使用原本样本中的一项来当做验证资料， 而剩余的则留下来当做训练资料。 这个步骤一直持续到每个样本都被当做一次验证资料。 事实上，这等同于 K-fold 交叉验证是一样的，其中K为原本样本个数。 在某些情况下是存在有效率的演算法，如使用kernel regression 和Tikhonov regularization。

##### 2-5-1-1 用StratifiedKFold实现十折交叉划分
将原始数据进行十折交叉划分，分成十份训练集和测试集并保存。

##### 2.3.2 利用十折交叉验证调参


In [ ]:
##### 2-5-1-1 用StratifiedKFold实现十折交叉划分     
## 将原始数据进行十折交叉划分，分成十份训练集和测试集并保存。

import pandas as pd
from sklearn.model_selection import StratifiedKFold

print('starting K10 cross-validation data split:')
train_df = pd.read_csv('TRAIN_DATA/train_set.csv', sep='\t') 

skf = StratifiedKFold(n_splits=10)     ## //分层采样，确保分出来的训练集和测试集中各类别样本的比例与原始数据集中相同。

for n_fold, (tr_idx, val_idx) in enumerate(skf.split(train_df['text'],train_df['label'])):
    print(f'the {n_fold} data split ...')
    tr_x, tr_y, val_x, val_y = train_df['text'].iloc[tr_idx], train_df['label'][tr_idx], train_df['text'].iloc[val_idx], train_df['label'][val_idx]
    tr_y = '__label__' + tr_y.astype(str)
    traindata = pd.DataFrame(list(zip(tr_x.values, tr_y.values)))
    traindata.to_csv(f'fasttext_skf10_datasplit/train_split{n_fold}.csv', index=None, header=['text', 'label_ft'], sep='\t')

    testdata = pd.DataFrame(list(zip(val_x.values, val_y.values)))
    testdata.to_csv(f'fasttext_skf10_datasplit/test_split{n_fold}.csv', index=None, header=['text', 'label'], sep='\t')


In [ ]:
##### 2.3.2 利用十折交叉验证调参

print('starting K10 cross-validation training:')
val_f1=[]

for n_fold in range(10):
    model = fasttext.train_supervised(f'fasttext_skf10_datasplit/train_split{n_fold}.csv', lr=1.0, wordNgrams=2,
                                      verbose=2, minCount=1, epoch=25, loss='softmax')
    val_df = pd.read_csv(f'fasttext_skf10_datasplit/test_split{n_fold}.csv', sep='\t')

    val_pred = [model.predict(x)[0][0].split('__')[-1] for x in val_df['text']]
    val_f1.append(f1_score(val_df['label'].values.astype(str), val_pred, average='macro'))
    print(f'the f1_score of {n_fold} training is:', val_f1[n_fold])

    
print()
print('The average f1_score is', sum(val_f1)/len(val_f1))

##  0.9303210475140224



#### 本章小结
   本章介绍了FastText的原理和基础使用，并进行相应的实践。然后介绍了通过10折交叉验证划分数据集。



#### 本章作业
   《1》阅读FastText的文档，尝试修改参数，得到更好的分数
   《2》基于验证集的结果调整超参数，使得模型性能更优
   

#### 参考案例
参考1地址 https://blog.csdn.net/qq_30481139/article/details/107582689    ## 自定义构造了训练，验证，预测的整个迭代过程 class TrainFunc()
参考2地址 